In [118]:
import time
from binance.client import Client
from binance.enums import *
import ccxt
import pandas as pd
import time
import hashlib
import hmac
import requests

API_KEY = 'f3aabfad7e741c38069d7c23729aea81bfe19e394f8f339f8c57cc92c77b83f0'
API_SECRET = 'd9fc0a87fa7aefe680bef2166f97e4880a00db95a7c6bdb609116dbf8e634f6b'

client = Client(API_KEY, API_SECRET, testnet=True)

In [118]:
def set_leverage(symbol, leverage, leverage_type):
    client.futures_change_leverage(symbol=symbol, leverage=leverage)


In [119]:
def trading_bot_1_margin(symbol, balance, profit_percent, current_price, current_date):
    trades = []
    holding = 0
    last_buy_balance = 0
    filename = 'bot_1_margin_trades_output.xlsx'

    try:
        df_trades = pd.read_excel(filename)
        last_trade = df_trades.iloc[-1]
        holding = last_trade['Holding']
    except FileNotFoundError:
        pass

    if holding == 0:
        buy_price = current_price
        last_buy_balance = balance
        quantity = balance / buy_price
        holding = quantity
        client.create_margin_order(
            symbol=symbol,
            side=Client.SIDE_BUY,
            type=Client.ORDER_TYPE_LIMIT,
            timeInForce=Client.TIME_IN_FORCE_GTC,
            quantity=quantity,
            price=buy_price
        )
        trades.append([symbol, 'Buy', current_date, buy_price, quantity, holding, balance, 0, 0])
    elif holding > 0:
        current_value = holding * current_price
        profit = (current_value - last_buy_balance) / last_buy_balance * 100

        if profit >= profit_percent:
            # Create a margin order
            client.create_margin_order(
                symbol=symbol,
                side=Client.SIDE_SELL,
                type=Client.ORDER_TYPE_LIMIT,
                timeInForce=Client.TIME_IN_FORCE_GTC,
                quantity=holding,
                price=current_price
            )
            trades.append([symbol, 'Sell', current_date, current_price, holding, 0, current_value, profit, 0])
            holding = 0
        else:
            trades.append([symbol, 'Hold', current_date, current_price, holding, holding, current_value, profit, 0])

    final_balance = df_trades['Balance'].iloc[-1]
    df = pd.DataFrame(trades, columns=['Symbol', 'Position', 'Date', 'Open Price', 'My Holding', 'Trade Holding', 'Balance', 'Profit', 'Initial Balance Profit'])
    df.to_excel('bot_1_trades.xlsx', index=False)
    return final_balance, trades

In [ ]:
def generate_signal(prev_close, current_close):
    if current_close >= prev_close:
        return 1
    elif current_close < prev_close:
        return -1

def trading_bot_2(symbol, leverage, balance, prev_close, current_close, current_date, profit_percent):
    signal = generate_signal(prev_close, current_close)

    trades = []
    holding = 0
    open_position_long = False
    open_position_short = False
    open_price = current_close
    hold_count = 0
    
    if os.path.exists('bot_2_trades.xlsx'):
        df = pd.read_excel('trades.xlsx')
        last_trade = df.iloc[-1]
        position = last_trade['Position']
        holding = last_trade['Holding']
        if position == 'Long' or position == 'Hold Long':
            open_position_long = True
        elif position == 'Short' or position == 'Hold Short':
            open_position_short = True

    if open_position_long:
        # if balance == 0:
        #     trades.append([symbol, 'Insufficient Balance', current_date, open_price, 0, 0, 0, 0])
        current_profit = (holding * open_price) - balance / balance * 100
        if(current_profit >= profit_percent):
            balance = holding * open_price
            trades.append([symbol, 'Take Profit Long', current_date, open_price, holding, 0, balance, current_profit])
            open_position_long = False
            hold_count = 0
            holding = 0
        else:
            hold_count = 1 + hold_count
            trades.append([symbol, 'Hold Long', current_date, open_price, holding, holding, balance, current_profit])
    elif open_position_short:
        if balance == 0:
            trades.append([symbol, 'Insufficient Balance', current_date, open_price, 0, 0, 0, 0])
        current_profit = (holding * open_price) - balance / balance * 100
        if(-1 * current_profit >= profit_percent):
            balance = balance + (holding * open_price - balance) * -1
            trades.append([symbol, 'Take Profit Short', current_date, open_price, holding, 0, balance, current_profit])
            open_position_short = False
            hold_count = 0
            holding = 0
        else:
            hold_count = 1 + hold_count
            trades.append([symbol, 'Hold Short', current_date, open_price, holding, holding, balance, current_profit])
    else:
        if signal == 1:
            open_position_long = True
            holding = balance / open_price
            client.create_order(
                symbol=symbol,
                side=Client.SIDE_BUY,
                type=Client.ORDER_TYPE_LIMIT,
                timeInForce=Client.TIME_IN_FORCE_GTC,
                quantity=holding,
                price=open_price
            )
            trades.append([symbol, 'Long', current_date, open_price, holding, holding, balance, 0])
        elif signal == -1:
            open_position_short = True
            holding = balance / open_price
            client.create_order(
                symbol=symbol,
                side=Client.SIDE_SELL,
                type=Client.ORDER_TYPE_LIMIT,
                timeInForce=Client.TIME_IN_FORCE_GTC,
                quantity=holding,
                price=open_price
            )
            trades.append([symbol, 'Short', current_date, open_price, holding, holding, balance, 0])
        else:
            trades.append([symbol, 'NO SIGNAL', current_date, open_price, 0, holding, balance, 0])

    final_balance = trades[-1][-2]
    
    df = pd.DataFrame(trades, columns=['Symbol', 'Position', 'Date', 'Open Price', 'My Holding', 'Trade Holding', 'Balance', 'Profit', 'Initial Balance Profit'])
    df.to_excel('bot_2_trades.xlsx', index=False)
    return final_balance

In [ ]:
def generate_trading_bot_3_signals(data):
    data['Signal'] = data.apply(generate_signal, axis=1)
    return data

def generate_signal(pullback_period, current_price, ema_200, ribbon, rsi):
    buy_condition_1 = current_price > ema_200
    buy_condition_2 = all(i > ema_200 for i in ribbon)
    buy_condition_3 = all(all(i > j for j in pullback_period) for i in ribbon)
    buy_condition_4 = rsi <= 40

    short_condition_1 = current_price < ema_200
    short_condition_2 = all(i < ema_200 for i in ribbon)
    short_condition_3 = all(all(i < j for j in pullback_period) for i in ribbon)
    short_condition_4 = rsi >= 60

    if buy_condition_1 and buy_condition_2 and buy_condition_3 and buy_condition_4:
        return 1
    elif short_condition_1 and short_condition_2 and short_condition_3 and short_condition_4:
        return 0
    else:
        return 2
    
def trading_bot_3(symbol, current_price, balance, current_date, profit_percent, pullback_period, ema_200, ribbon, rsi):
    # Generate Signal
    signal = generate_signal(pullback_period, current_price, ema_200, ribbon, rsi)
    trades = []
    holding = 0
    last_buy_balance = 0
    holding_count = 0
    
    if os.path.exists('trades.xlsx'):
        df = pd.read_excel('trades.xlsx')
        last_trade = df.iloc[-1]
        mode = last_trade['Position']
        holding = last_trade['Holding']
        
    if holding > 0:
        current_value = holding * current_price
        profit = (current_value - last_buy_balance) / last_buy_balance * 100

        if profit >= profit_percent:
            balance = current_value
            holding = 0
            last_buy_balance = current_value
            profit_to_initial_balance = balance - balance / balance * 100
            if mode == 'Long':
                trades.append([symbol, 'Sell', current_date, current_price, holding, balance, profit, profit_to_initial_balance])
            else:
                trades.append([symbol, 'Buy', current_date, current_price, holding, balance-profit_percent, profit, profit_to_initial_balance])
        else:
            holding_count = holding_count + 1
            trades.append([symbol, 'Hold', current_date, current_price, holding, current_value, profit, 0])

    if holding == 0:
        if signal == 1:
            if holding == 0:
                long_price = current_price
                real_balance = balance - balance * 100
                last_buy_balance = real_balance
                holding = real_balance / long_price
                trades.append([symbol, 'Long', current_date, long_price, holding, holding, 0, 0])
        elif signal == 0:
            if holding == 0:
                short_price = current_price
                balance = holding * current_price
                holding = 0
                profit_to_initial_balance = balance - balance / balance * 100
                trades.append([symbol, 'Short', current_date, short_price, holding, holding, 0, profit_to_initial_balance])

    final_balance = trades[-1][-3]
    
    df = pd.DataFrame(trades, columns=['Symbol', 'Position', 'Date', 'Open Price', 'My Holding', 'Trade Holding', 'Balance', 'Profit', 'Initial Balance Profit'])
    df.to_excel('bot_3_trades.xlsx', index=False)
    return final_balance

In [120]:
api_key = 'f3aabfad7e741c38069d7c23729aea81bfe19e394f8f339f8c57cc92c77b83f0'
api_secret = 'd9fc0a87fa7aefe680bef2166f97e4880a00db95a7c6bdb609116dbf8e634f6b'
symbol = 'BTCUSDT'
leverage = 2
leverage_type = 'CROSSED'
profit_percent = 5
balance = client.futures_account(version=2)
client = Client(api_key, api_secret, testnet=True)
set_leverage(client, symbol, leverage)
print(balance)

BinanceAPIException: APIError(code=-5000): GET /fapi/v1/account is retired, please use GET /fapi/v2/account

In [ ]:
while balance > 0:
    balance = trading_bot_1_futures(symbol, leverage, leverage_type, balance, profit_percent)
    print(f"Current Balance: {balance} USDT")

    # Introduce a delay (e.g., 15 minutes) to avoid API rate limits
    time.sleep(15 * 60)

print("Trading stopped. Balance is zero.")